In [1]:
import os, sys, torch
import numpy as np
import scipy as sp
import multiprocessing as mp
import neuropythy as ny
import pandas
import matplotlib.pyplot as plt
import ipyvolume as ipv
import pythreejs

In [2]:
%matplotlib inline

In [3]:
participant_ids = [
    "sub-wlsubj001", "sub-wlsubj004", "sub-wlsubj006", "sub-wlsubj007",
    "sub-wlsubj014", "sub-wlsubj019", "sub-wlsubj023", "sub-wlsubj042",
    "sub-wlsubj045", "sub-wlsubj046", "sub-wlsubj055", "sub-wlsubj056",
    "sub-wlsubj057", "sub-wlsubj062", "sub-wlsubj064", "sub-wlsubj067",
    "sub-wlsubj071", "sub-wlsubj076", "sub-wlsubj079", "sub-wlsubj081",
    "sub-wlsubj083", "sub-wlsubj084", "sub-wlsubj085", "sub-wlsubj086",
    "sub-wlsubj087", "sub-wlsubj088", "sub-wlsubj090", "sub-wlsubj091",
    "sub-wlsubj092", "sub-wlsubj095", "sub-wlsubj104", "sub-wlsubj105",
    "sub-wlsubj109", "sub-wlsubj114", "sub-wlsubj115", "sub-wlsubj116",
    "sub-wlsubj117", "sub-wlsubj118", "sub-wlsubj120", "sub-wlsubj122",
    "sub-wlsubj126"]
participant_ids = np.array(participant_ids)

In [4]:
def load_surf(pid, h, surface='white', as_fsaverage=True,
              dataset_path='/data/openneuro/ds003787',
              as_hemi=False):
    "Loads a subject's hemisphere from the NYU Retinotopy Dataset."
    # Prepare some paths.
    der_path = os.path.join(dataset_path, 'derivatives')
    prf_path = os.path.join(der_path, 'prfanalyze-vista', pid, 'ses-nyu3t01')
    roi_path = os.path.join(der_path, 'ROIs', pid)
    sub_path = os.path.join(der_path, 'freesurfer', pid)
    bay_path = os.path.join(der_path, 'bayesian_inference_maps', pid)
    # Load the subject.
    sub = ny.freesurfer_subject(sub_path, check_path=False)
    # Load the hemisphere properties.
    hem = sub.hemis[h]
    srf = hem.surface(surface)
    hem = hem.with_prop(
        prf_x=ny.load(f'{prf_path}/{h}.x.mgz'),
        prf_y=ny.load(f'{prf_path}/{h}.y.mgz'),
        prf_cod=ny.load(f'{prf_path}/{h}.vexpl.mgz'),
        visual_area=ny.load(f'{roi_path}/{h}.ROIs_V1-4.mgz'),
        x=srf.coordinates[0],
        y=srf.coordinates[1],
        z=srf.coordinates[2],
        inf_polar_angle=ny.load(f'{bay_path}/{h}.inferred_angle.mgz'),
        inf_eccentricity=ny.load(f'{bay_path}/{h}.inferred_eccen.mgz'),
        inf_visual_area=ny.load(f'{bay_path}/{h}.inferred_varea.mgz'))
    if not as_fsaverage:
        if as_hemi: return hem
        return hem.surface(surface)
    # Load the Freesurfeer Subject
    fsaverage = ny.freesurfer_subject('fsaverage')
    fshem = fsaverage.hemis[h]
    # Interpolate the properties over to the fsaverage.
    fs_props = hem.interpolate(fshem, hem.properties)
    fshem = fshem.with_prop(fs_props)
    fssrf = fshem.surface(surface)
    fssrf = fssrf.copy(coordinates=[fs_props['x'], fs_props['y'], fs_props['z']])
    (ang,ecc) = ny.as_retinotopy({'lon':fs_props['prf_x'], 'lat':-fs_props['prf_y']},
                                 'visual')
    fssrf = fssrf.with_prop(polar_angle=ang, eccentricity=ecc,
                            cod=fs_props['prf_cod'])
    return fssrf    

In [5]:
def safesqrt(x):
    ii = (x != 0)
    y = x.clone()
    y[ii] = torch.sqrt(x[ii])
    return y
def angmod(theta, hva=0.5, vma=0.5):
    from numpy import pi
    from torch import sin, cos, sign
    hvpart = hva * cos(2 * theta)
    thsin  = sin(theta)
    ulpart = vma * sign(thsin) * thsin**2
    return 1.0 + 0.5*(hvpart - ulpart)
def vismodel(hem, label=1, max_eccen=9, hh91_c2=0.75, fsign=-1):
    import torch
    # Get the submesh we operate on.
    h = hem.chirality
    va = hem.prop('visual_area')
    lbl = (va == label)
    if ny.is_topo(hem):
        mesh0 = hem.surface('midgray')
    else:
        mesh0 = hem
    mesh = mesh0.submesh(lbl)
    # Get the faces in the submesh.
    (a,b,c) = np.array(mesh.tess.faces)
    # And the surface areas.
    cortex_sarea = torch.tensor(mesh.face_areas)
    cortex_sarea_tot = torch.sum(cortex_sarea)
    # And the edge lengths.
    sxy = torch.tensor(mesh0.coordinates)
    ab_surf_elen = safesqrt(torch.sum((sxy[:,b] - sxy[:,a])**2, axis=0))
    bc_surf_elen = safesqrt(torch.sum((sxy[:,c] - sxy[:,b])**2, axis=0))
    ca_surf_elen = safesqrt(torch.sum((sxy[:,a] - sxy[:,c])**2, axis=0))
    # Calculate the H&H c1 value for this surface area.
    c2_maxecc = torch.tensor(hh91_c2 + max_eccen)
    den = np.pi * (torch.log(c2_maxecc / hh91_c2) - max_eccen / c2_maxecc)
    c1 = safesqrt(cortex_sarea_tot / den)
    # We also want to know what vertices are along the outer edge
    (u,v) = mesh0.tess.edges
    (uii,vii) = [np.where((va[u] == label) & (va[v] != label))[0],
                 np.where((va[v] == label) & (va[u] != label))[0]]
    ii_maxecc = [u[uii[va[v[uii]] == 0]], v[vii[va[u[vii]] == 0]]]
    ii_hm     = [u[uii[va[v[uii]] != 0]], v[vii[va[u[vii]] != 0]]]
    ii_maxecc = np.unique(np.concatenate(ii_maxecc))
    ii_hm     = np.unique(np.concatenate(ii_hm))
    # However, we want u and v to refer to mesh, not mesh0, below.
    (u,v) = mesh.tess.edges
    # Here's the basic model:
    def _loss(xy, save_fsign=None):
        # For each triangle, we need to know the cortical magnification.
        # For this, we first calculate the visual surface area of each face.
        (a_xy, b_xy, c_xy) = [xy[:,k] for k in (a, b, c)]
        ab_xy = b_xy - a_xy
        bc_xy = c_xy - b_xy
        ca_xy = a_xy - c_xy
        # We want to take the cross-product of ab x -ca; however, V1 has a
        # negative field-sign, so we can ignore the minus-sign if we want a
        # positive field-sign.
        visual_fsign = fsign*torch.sign(ab_xy[0]*ca_xy[1] - ca_xy[0]*ab_xy[1])
        if save_fsign is not None: save_fsign[0] = visual_fsign
        (ablen, bclen, calen) = [safesqrt(torch.sum(uu**2, axis=0))
                                 for uu in (ab_xy, bc_xy, ca_xy)]
        s = (ablen + bclen + calen) / 2
        visual_sarea = safesqrt(s*(s - ablen)*(s - bclen)*(s - calen))
        visual_sarea = visual_fsign * visual_sarea
        # Next predict the Horton and Hoyt (1991) cortical magnification for
        # each triangle based on its average eccentricity and its cortical
        # surface area.
        face_xy = (a_xy + b_xy + c_xy) / 3
        face_eccen = safesqrt(torch.sum(face_xy**2, axis=0))
        face_theta = torch.atan2(face_xy[1], face_xy[0])
        hh91_cmag = (c1 / (hh91_c2 + face_eccen))**2
        hh91_cmag = hh91_cmag * angmod(face_theta)
        hh91_visual_sarea = cortex_sarea / hh91_cmag
        # The loss is the difference between the predicted and the measured
        # visual surface area.
        dd = visual_sarea - hh91_visual_sarea
        ii = dd < 0
        dd2 = dd**2
        dd2[ii] -= dd[ii] + dd[ii]**3
        dd = torch.mean(dd)
        # Similar idea for the edges.
        hh91_lincmag = safesqrt(hh91_cmag)
        ee = ((ablen*visual_fsign - ab_surf_elen/hh91_lincmag)**2 +
              (bclen*visual_fsign - bc_surf_elen/hh91_lincmag)**2 +
              (calen*visual_fsign - ca_surf_elen/hh91_lincmag)**2)
        ee = torch.mean(ee)
        # We also add in the outer difference.
        outer_ecc = safesqrt(torch.sum(xy[:,ii_maxecc]**2, axis=0))
        outer_hm  = xy[0, ii_hm]
        oo = torch.mean((outer_ecc - (max_eccen+2))**2) + torch.mean(outer_hm**2)
        return (dd, ee, oo)
    # Also, get the initial starting points.
    (x0,y0) = ny.as_retinotopy(
        ny.retinotopy_data(hem, 'inf_'),
        'geographical')
    if h == 'rh': x0 = -x0
    x0 = torch.tensor(x0.astype(float))
    y0 = torch.tensor(y0.astype(float))
    xy0 = torch.vstack([x0[None,:], y0[None,:]])
    xy0[:, ~lbl] = np.nan
    # Also, get map-based starting points
    fmap = ny.to_flatmap('occipital_pole', hem)
    ii = np.isin(fmap.labels, mesh.labels)
    tmp = np.array([fmap.coordinates[0,ii], -fmap.coordinates[1,ii]])
    xy1 = torch.zeros_like(xy0)
    xy1[:,mesh.labels] = torch.tensor(np.array(tmp))
    return (_loss, mesh, xy0, xy1)

In [6]:
prfmodel_plan = [
    dict(steps=1000, pw=1, ew=1, lr=0.20, lr_decay=0),
    dict(steps=1000, pw=1, ew=1, lr=0.15, lr_decay=0),
    dict(steps=1000, pw=1, ew=1, lr=0.10, lr_decay=0.01)]
def calc_prfmodel(pid, h, plan=prfmodel_plan):
    hem = load_surf(pid, h,
                    as_fsaverage=False, as_hemi=True)
    # Get the loss function and the initial PRF solutions.
    (lossfn, mesh, xy0, xy1) = vismodel(hem)
    # Here's what we minimize.
    xy = xy0.clone().detach()
    ii = torch.any(~torch.isfinite(xy[:,mesh.labels]), axis=0)
    xy[:,mesh.labels[ii]] = 0
    #xy[0] -= torch.min(xy[0])
    xy = xy.requires_grad_(True)
    for (roundno,plan) in enumerate(plan):
        steps = plan.get('steps', 100)
        lr = plan.get('lr', 0.1)
        lr_decay = plan.get('lr_decay', 0)
        ww = plan.get('pw', 1)
        ew = plan.get('ew', 1)
        opt = torch.optim.Adagrad([xy], lr=lr, lr_decay=lr_decay)
        for step in range(steps):
            def closure():
                opt.zero_grad()
                (dd,ee,oo) = lossfn(xy)
                ll = dd if torch.isfinite(dd) else 1.0e10
                if ew > 0: ll = ll + ee*ew
                if ww > 0: ll = ll + oo*ww
                ll.backward()
                return ll
            opt.step(closure)
    return xy
def save_prfmodel(flnm, pid, h):
    if os.path.isfile(flnm): return flnm
    xy = calc_prfmodel(pid, h)
    xy = xy.detach().numpy()
    return ny.save(flnm, xy.T)
def load_prfmodel(pid, h):
    flnm = os.path.join("/data/nyuretinotopy_morph/prfmodels",
                        f"{h}.{pid}.mgz")
    if not os.path.isfile(flnm):
        save_prfmodel(flnm, pid, h)
    return ny.load(flnm)
def ensure_prfmodels(pid):
    a = load_prfmodel(pid, 'lh')
    b = load_prfmodel(pid, 'rh')
    return (a.shape, b.shape)

In [7]:
def cos_tr(x):
    y = (1 - np.cos(np.pi*2*(x-0.25)))/2
    y[x < 0.25] = 0
    y[x > 0.75] = 1
    return y
frame_rate = 30
step_duration = 2
frames_per_step = frame_rate * step_duration
step_t0 = np.arange(0, frames_per_step) / (frames_per_step - 1)
step_t = cos_tr(step_t0)
vfmesh = ny.vision.visual_field_mesh(max_eccentricity=9, resolution=0.5)

In [15]:
def calc_vfield(pid):
    hs = {}
    cms = {}
    for h in ['lh','rh']:
        hem = load_surf(pid, h, as_fsaverage=False, as_hemi=True)
        mdl = load_prfmodel(pid, h)
        (x,y) = mdl.T
        ii = np.isfinite(x) & np.isfinite(y)
        vfm = hem.white_surface.submesh(ii)
        # Adjust the coordnates.
        vfm = vfm.copy(coordinates=mdl[vfm.labels])
        hs[h] = vfm
        # Calculate vmag.
        cms[h] = ny.vision.areal_cmag(vfm, retinotopy='prf_')
    # Sample properties over to the visual field mesh
    ps = {}
    for (h,m) in hs.items():
        u = m.interpolate(vfmesh, ['prf_x','prf_y'])
        u = dict(x=u[0], y=u[1], cmag=cms[h](*vfmesh.coordinates))
        ps[h] = u
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ps = {k: np.nanmean([ps['lh'][k], ps['rh'][k]], axis=0)
              for k in ps['lh'].keys()}
        neis = vfmesh.tess.indexed_neighborhoods
        (found,fillno) = (True, 0)
        while found and fillno < 4:
            found = False
            fillno += 1
            for (k,v) in ps.items():
                ii = np.where(~np.isfinite(v))[0]
                if len(ii) == 0: continue
                found = True
                v[ii] = [np.nanmean(v[list(neis[k])]) for k in ii]
        #for (k,v) in ps.items():
        #    ii = np.where(~np.isfinite(v))[0]
        #    if len(ii) == 0: continue
        #    v[ii] = 0
        return np.array([ps['x'],ps['y'],ps['cmag']])
def save_vfield(flnm, pid, overwrite=False):
    if not overwrite and os.path.isfile(flnm): return flnm
    xy = calc_vfield(pid)
    return ny.save(flnm, xy)
def load_vfield(pid, overwrite=False):
    flnm = os.path.join("/data/nyuretinotopy_morph/prfmodels",
                        f"{pid}_vfmesh.mgz")
    if overwrite or not os.path.isfile(flnm):
        save_vfield(flnm, pid, overwrite=overwrite)
    return ny.load(flnm)
def ensure_vfield(pid):
    try:
        return load_vfield(pid).shape
    except Exception as e:
        return e

In [28]:
def _plot_vfield(x, y, ax):
    ang = np.mod(90 - 180/np.pi*np.arctan2(y, x) + 180, 360) - 180
    return ny.cortex_plot(vfmesh, {'polar_angle':ang},
                          underlay='w', axes=ax)
def _plot_cmag(cmag, ax):
    cmag = np.array(cmag)
    cmag[~np.isfinite(cmag)] = 1e-10
    cmag = np.log2(cmag)
    cm = 'hot'
    return ny.cortex_plot(vfmesh, color=cmag, cmap=cm,
                          vmin=np.log2(2.0), vmax=np.log2(1024.0),
                          underlay='0.5', axes=ax)
def save_vfframe(pid1, pid2, frame0, w=None, figsize=(2,2), dpi=240,
                 overwrite=False):
    (x1,y1,cmag1) = load_vfield(pid1)
    (x2,y2,cmag2) = load_vfield(pid2)
    if w is None:
        w = np.linspace(0,1,frames_per_step)
        w = cos_tr(w)
    (fig,ax) = plt.subplots(1,1, figsize=figsize, dpi=dpi)
    fig.subplots_adjust(0,0,1,1,0,0)
    for (f,w) in enumerate(w):
        if w == 0:   f = frame0
        elif w == 1: f = frame0 + frames_per_step - 1
        else:        f += frame0
        flnm = os.path.join('/data/nyuretinotopy_morph/frames_model',
                            'frame%05d.png' % (f,))
        if not overwrite and os.path.isfile(flnm): continue
        x = x1*(1-w) + x2*w
        y = y1*(1-w) + y2*w
        cmag = cmag1*(1-w) + cmag2*w
        ax.clear()
        #pp = _plot_vfield(x, y, ax)
        pp = _plot_cmag(cmag, ax)
        ax.set_xlim([-9.1,9.1])
        ax.set_ylim([-9.1,9.1])
        ax.axis('off')
        # Save the figure.
        plt.savefig(flnm, bbox_inches='tight')
    return fig
def ensure_vfframe(tup):
    fig = save_vfframe(*tup, overwrite=True)
    plt.close(fig)
    return None

In [11]:
njobs = len(participant_ids)
nproc = 12

for ii in range(0, njobs, nproc):
    jj = min(njobs, ii + nproc)
    print(ii, "of", njobs)
    with mp.Pool(jj - ii + 1) as pool:
        pool.map(ensure_vfield, participant_ids[ii:jj])

0 of 41
12 of 41
24 of 41
36 of 41


In [29]:
nproc = 12

pids = participant_ids
jobs = [(pid1,pid2,ii*frames_per_step)
        for (ii,(pid1,pid2)) in enumerate(zip(pids,np.roll(pids,-1)))]
njobs = len(jobs)

for ii in range(0, njobs, nproc):
    jj = min(njobs, ii + nproc)
    print(ii, "of", njobs)
    with mp.Pool(jj - ii + 1) as pool:
        pool.map(ensure_vfframe, jobs[ii:jj])

0 of 41
12 of 41
24 of 41
36 of 41


In [36]:
def postproc_frame(fno):
    bp = '/data/nyuretinotopy_morph'
    fl1 = os.path.join(bp, 'frames', 'frame%05d.png' % fno)
    fl2 = os.path.join(bp, 'frames_model', 'frame%05d.png' % fno)
    flo = os.path.join(bp, 'frames_final', 'frame%05d.png' % fno)
    if os.path.isfile(flo): return flo
    im1 = plt.imread(fl1)
    im2 = plt.imread(fl2)
    (r,c) = im2.shape[:2]
    hc = int(c/2)
    c0 = int(im1.shape[1]/2) - hc
    subim = im1[1180:(1180+r),c0:(c0+c),:]
    fixim = np.flip(im2[:,:,:3], axis=0)
    ii = np.where(~np.all(fixim == 1, axis=2))
    subim[ii] = fixim[ii]
    return plt.imsave(flo, im1)
def postproc_job(tup):
    (i0,skip) = tup
    for k in range(i0, 2460, skip):
        postproc_frame(k)
    return None

In [ ]:
(fig,ax) = plt.subplots(1,1, figsize=(1,5), dpi=288)

im = np.transpose([np.linspace(2,1024,300)]*30)
ax.imshow(im,
          cmap='hot',
          vmin=2, vmax=1024)
ax.invert_yaxis()
ax.set_yticks([0,100,200,300])
ax.set_yticklabels([2,16,128,1024])

In [37]:
nproc = 16
with mp.Pool(nproc) as pool:
    pool.map(postproc_job, enumerate([nproc]*nproc))